# 🏦 Projeto Didático: Predição de Risco de Crédito (ERP Fênix)

Neste notebook, demonstraremos como um modelo de Machine Learning pode prever o comportamento de parcelas **atualmente em aberto** baseando-se no comportamento histórico dos clientes.

### Conceitos Chave:
1. **Features de Perfil:** Dados do cliente (limite, tempo de casa).
2. **Evitando Data Leakage:** Não usaremos dados que só existem após a baixa da parcela.
3. **Separação Treino/Aplicação:** Treinamos com o histórico e aplicamos nas parcelas abertas.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore') # suprime os avisos (warnings) durante a execução do código.
plt.style.use('seaborn-v0_8-darkgrid')

# Configuração da Conexão
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
    'database': 'fenix',
    'port': 3306
}

engine = create_engine(f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
print("✅ Conexão preparada!")

✅ Conexão preparada!


### 1️⃣ Extração de Dados (Com Join de Clientes)
Buscamos dados da parcela e o perfil do cliente que a gerou.

In [3]:
# A consulta foi desenhada para extrair tudo o que importa para prever se um cliente vai pagar uma parcela futura, 
# baseando-se no seu perfil e nas características do contrato. 
query = """
SELECT 
    pr.id,
    pr.valor as valor_parcela,
    pr.numero_parcela,
    lr.quantidade_parcela as total_parcelas_carne,
    c.limite_credito,
    c.desde as cliente_desde,
    c.taxa_desconto as desconto_cliente,
    sp.descricao as status_atual,
    DATEDIFF(pr.data_vencimento, pr.data_emissao) as prazo_pagamento_dias
FROM fin_parcela_receber pr
JOIN fin_lancamento_receber lr ON pr.id_fin_lancamento_receber = lr.id
JOIN cliente c ON lr.id_cliente = c.id
JOIN fin_status_parcela sp ON pr.id_fin_status_parcela = sp.id
"""

df_raw = pd.read_sql(query, engine) # Carrega os dados da consulta SQL em um DataFrame do Pandas

# Feature Engineering básica - criando novas colunas a partir das existentes
df_raw['cliente_desde'] = pd.to_datetime(df_raw['cliente_desde']) # converte para datetime
df_raw['tempo_casa_dias'] = (pd.Timestamp.now() - df_raw['cliente_desde']).dt.days # calcula o tempo como cliente em dias
df_raw['percentual_limite_ocupado'] = df_raw['valor_parcela'] / df_raw['limite_credito'] # calcula o percentual do limite de crédito ocupado pela parcela

print(f"📦 Total de registros carregados: {len(df_raw)}")
df_raw.head(10) # mostra as primeiras linhas do DataFrame carregado

📦 Total de registros carregados: 2822


,id,valor_parcela,numero_parcela,total_parcelas_carne,limite_credito,cliente_desde,desconto_cliente,status_atual,prazo_pagamento_dias,tempo_casa_dias,percentual_limite_ocupado
0,11,2500.000000,1,2,2500.0,2025-02-25,10.0,Quitado,767,303,1.000000
1,12,2500.000000,2,2,2500.0,2025-02-25,10.0,Quitado,795,303,1.000000
2,24,2420.416667,1,3,2500.0,2025-02-25,10.0,Aberto,-826,303,0.968167
3,25,2420.416667,2,3,2500.0,2025-02-25,10.0,Aberto,-819,303,0.968167
4,26,2420.416667,3,3,2500.0,2025-02-25,10.0,Aberto,-812,303,0.968167
5,96,102.871921,1,10,2500.0,2025-02-25,10.0,Quitado,30,303,0.041149
6,97,102.871921,2,10,2500.0,2025-02-25,10.0,Aberto,60,303,0.041149
7,98,102.871921,3,10,2500.0,2025-02-25,10.0,Aberto,90,303,0.041149
8,99,102.871921,4,10,2500.0,2025-02-25,10.0,Aberto,120,303,0.041149
9,100,102.871921,5,10,2500.0,2025-02-25,10.0,Aberto,150,303,0.041149


### 2️⃣ Separação: O que o modelo estuda vs. O que o modelo prevê
O modelo não pode ver as parcelas 'Abertas' durante o treino. Evitando o data leakage (vazamento de informação)

In [4]:
# 1. Histórico: Tudo que já foi resolvido (Pago ou Vencido)
# Separando apenas parcelas cujo destino já é conhecido
df_historico = df_raw[df_raw['status_atual'].str.contains('Quitado|Vencido', case=False, na=False)].copy()
df_historico['target'] = df_historico['status_atual'].apply(lambda x: 1 if 'Quitado' in x else 0) # 1 = Quitado, 0 = Vencido - classificação supervisionada

# 2. Futuro: Tudo que está em aberto hoje
# Essas parcelas ainda não acabaram — vamos pedir um palpite ao modelo
# case=False torna a busca case insensitive; na=False evita problemas com valores nulos
df_abertas = df_raw[df_raw['status_atual'].str.contains('Aberto', case=False, na=False)].copy()

print(f"📚 Registros para Treino: {len(df_historico)}")
print(f"🔮 Registros para Prever: {len(df_abertas)}")

# O modelo só aprende com parcelas que já tiveram um fim (quitadas ou vencidas) e é usado para estimar a chance de pagamento 
# das parcelas que ainda estão em aberto — exatamente como no mundo real.

📚 Registros para Treino: 504
🔮 Registros para Prever: 2318


### 3️⃣ Treinamento do Modelo
Usaremos as features de perfil para ensinar o computador.

In [5]:
from sklearn.ensemble import RandomForestClassifier # Importa o classificador Random Forest
from sklearn.model_selection import train_test_split # Importa a função para dividir os dados em treino e teste

# Define as features (variáveis independentes)
# Essas são as únicas informações que o modelo pode usar para fazer previsões
# valor_parcela:	Quanto o cliente precisa pagar
# limite_credito:	Capacidade financeira
# tempo_casa_dias:	Relacionamento com a empresa
# percentual_limite_ocupado:	Quanto do limite está comprometido
# prazo_pagamento_dias:	Tempo dado para pagar
features = ['valor_parcela', 'limite_credito', 'tempo_casa_dias', 'percentual_limite_ocupado', 'prazo_pagamento_dias'] 

# X é a tabela que o modelo enxerga (features), y é o que queremos prever (target)
X = df_historico[features].fillna(0) # Preenche valores nulos com 0
y = df_historico['target'] # Variável alvo - 1 para Quitado, 0 para Vencido

if len(df_historico) > 1: # Verifica se há dados suficientes para treino
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Divide os dados em treino e teste, 20% para teste
    
    model = RandomForestClassifier(n_estimators=100) # Cria o modelo Random Forest, que é robusto e eficaz para classificação - 100 árvores na floresta
    model.fit(X_train, y_train) # Treina o modelo, ajustando-o aos dados de treino
    
    print("✅ Modelo treinado com sucesso!")
else:
    print("⚠️ Dados insuficientes para treino. Adicione mais registros quitados/vencidos no banco.")

✅ Modelo treinado com sucesso!


### 4️⃣ A Hora da Verdade: Prevendo o Futuro
Agora pedimos ao modelo para olhar as parcelas 'Abertas' e calcular a probabilidade de pagamento.

In [6]:
if len(df_abertas) > 0:
    X_futuro = df_abertas[features].fillna(0) # Preenche valores nulos com 0 e usa as mesmas features do treino
    
    # Pegamos as probabilidades
    probs = model.predict_proba(X_futuro) # Retorna a probabilidade de cada classe (0 e 1)
    
    # Verificamos se o modelo conhece as duas classes (0 e 1)
    if probs.shape[1] > 1: # shape[1] → número de classes aprendidas
				# shape = (n, 2) - Modelo aprendeu pagar e não pagar
				# shape = (n, 1) - Modelo aprendeu só pagar ou só não pagar
        # Se ele conhece ambas, pegamos a probabilidade da classe 1 (Pago)
        df_abertas['probabilidade_pagamento'] = probs[:, 1] # : → todas as linhas, 1 → coluna da classe paga
    else:
        # Se ele só conhece uma classe, marcamos 100% ou 0% baseado no que ele aprendeu
        # Se o modelo só conhece um tipo de comportamento, ele assume que tudo será igual
        # Exemplo:
        # Só viu Quitado → 100%
        # Só viu Vencido → 0%
        unica_classe = model.classes_[0]
        df_abertas['probabilidade_pagamento'] = 1.0 if unica_classe == 1 else 0.0 # só vai acontecer se todas as parcelas forem QUITADAS ou VENCIDAS
    
    # Resultado Final
    resultado_exibicao = df_abertas[['id', 'valor_parcela', 'limite_credito', 'probabilidade_pagamento']].sort_values(by='probabilidade_pagamento')
    
    print("📊 PREVISÃO DAS PARCELAS EM ABERTO:")
    print(resultado_exibicao)
else:
    print("Não há parcelas abertas para prever.")

📊 PREVISÃO DAS PARCELAS EM ABERTO:
        id  valor_parcela  limite_credito  probabilidade_pagamento
81    1441     177.265525          2500.0                     0.09
80    1440     177.265525          2500.0                     0.09
83    1443     177.265525          2500.0                     0.09
82    1442     177.265525          2500.0                     0.09
86    1446     177.265525          2500.0                     0.09
...    ...            ...             ...                      ...
2800  2201     299.837839          7800.0                     1.00
2820  2739     113.058633          7800.0                     1.00
2821  2740     113.058633          7800.0                     1.00
2799  2200     299.837839          7800.0                     1.00
2792  1727     281.678305          7800.0                     1.00

[2318 rows x 4 columns]
